In [1]:
library(pheatmap)
library(plyr)

In [2]:
# Load integrals, peak ranges, and sample data
path <- "~/Documents/Projects/asf_interactions/data/"
integral_file <- 'integrals_20170323.txt'
integral_range_file <- 'coculture_peak_ranges.txt'
integrals = read.table(paste0(path,integral_file),header=FALSE,
                         sep = ",")
integral_ranges = read.table(paste0(path,integral_range_file),header=TRUE,
                             sep = ",")
master <- read.table(file=paste0(path,"merged_metadata.txt"),header=TRUE,sep='\t')

In [3]:
# set column names of integrals using integral_ranges met names
colnames(integrals) <- integral_ranges$met_name
mets = colnames(integrals)
# Set rownames using experiment_id. This should be ordered correctly.
integrals$experiment_id <- master$experiment_id
# merge so that classes and integrals are in one dataframe
all_data <- join(master,integrals,by="experiment_id")

In [4]:
# remove non-growing samples from the data
# 500 and 492 did not grow in experiment 3
# 502 did not grow in experiment 2 or 3
all_data = all_data[!(all_data$Run == 2 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('500',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('492',all_data$species)),]
# remove experiment 4 (3-species subset including only 356,361,519)
all_data = all_data[all_data$Run != "4",]

In [5]:
# replace negative values in integrals with 0.
all_data[,colnames(all_data) %in% mets] = 
        (apply(all_data[,colnames(all_data) %in% mets],2,function(x){x[x<0] = 0;x}))
# center the data by subtracting the mean of blanks
blanks = subset(all_data,species == '0')
blank_means = colMeans(blanks[,names(blanks) %in% mets])
centered_integrals <- sweep(all_data[,mets], 2, blank_means, `-`)
all_data[,mets] = centered_integrals

In [6]:
known_mets = mets[-grep("unknown", mets)]
monoculture_only = all_data[all_data$species %in% c('356','360','361','492','500','502','519'),]
# collapse columns into means
mono356 = colMeans(monoculture_only[monoculture_only$species == '356',known_mets])
mono360 = colMeans(monoculture_only[monoculture_only$species == '360',known_mets])
monoculture_for_plotting = rbind(mono356,mono360)
mono361 = colMeans(monoculture_only[monoculture_only$species == '361',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono361)
mono492 = colMeans(monoculture_only[monoculture_only$species == '492',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono492)
mono500 = colMeans(monoculture_only[monoculture_only$species == '500',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono500)
mono502 = colMeans(monoculture_only[monoculture_only$species == '502',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono502)
mono519 = colMeans(monoculture_only[monoculture_only$species == '519',known_mets])
monoculture_for_plotting = rbind(monoculture_for_plotting,mono519)
row.names(monoculture_for_plotting) = c('ASF356','ASF360','ASF361','ASF492','ASF500','ASF502','ASF519')
# try min/max scaling within each column
monoculture_for_plotting_maxnorm = sweep(monoculture_for_plotting,MARGIN=2,apply(monoculture_for_plotting,2,function(x) max(abs(x), na.rm = TRUE)), FUN="/")

In [7]:
# generate a heatmap of monoculture metabolomes using the centered, max/min normalized integrals
pheatmap(monoculture_for_plotting_maxnorm,filename='~/Documents/Projects/asf_interactions/results/monoculture_metabolome.jpg',width=9,height=3.2,cluster_rows=FALSE)

In [8]:
# Generate the same heatmap for mono and co-cultures for supplement... just do it manually.
nextdf = colMeans(all_data[all_data$species == '356,360',known_mets])
coculture_plotting = rbind(monoculture_for_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '356,361',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '356,492',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '356,500',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '356,502',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '356,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '360,361',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '360,492',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '360,500',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '360,502',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '360,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '361,492',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '361,500',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '361,502',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '361,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '492,500',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '492,502',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '492,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '500,502',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '500,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
nextdf = colMeans(all_data[all_data$species == '502,519',known_mets])
coculture_plotting = rbind(coculture_plotting,nextdf)
row.names(coculture_plotting) = c('ASF356','ASF360','ASF361','ASF492','ASF500','ASF502','ASF519',
                                        'ASF356+360','ASF356+361','ASF356+492','ASF356+500','ASF356+502','ASF356+519',
                                        'ASF360+361','ASF360+492','ASF360+500','ASF360+502','ASF360+519',
                                        'ASF361+492','ASF361+500','ASF361+502','ASF361+519',
                                        'ASF492+500','ASF492+502','ASF492+519',
                                        'ASF500+502','ASF500+519',
                                        'ASF502+519')
coculture_plotting_maxnorm = sweep(coculture_plotting,MARGIN=2,apply(coculture_plotting,2,function(x) max(abs(x), na.rm = TRUE)), FUN="/")

pheatmap(coculture_plotting_maxnorm,filename='~/Documents/Projects/asf_interactions/results/coculture_metabolome.jpg',width=9,height=6,cluster_rows=TRUE)